In [11]:
# Instalar dependencias sin matplotlib problemático
%pip install scikit-learn pandas numpy seaborn
import os

os.environ['MPLBACKEND'] = 'Agg'
import matplotlib
matplotlib.use('Agg', force=True)



4803.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


# 1. Carga de datos

In [12]:
import pandas as pd

DATA_PATH = "./processes2.csv"

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head()


Shape: (2095, 14)


,Unnamed: 0,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,seats,max_power (in bph),Mileage Unit,Mileage,Engine (CC)
0,0,Maruti,2014,450000,145500,Diesel,Individual,Manual,First Owner,5,74.00,kmpl,23.40,1248
1,2,Hyundai,2010,225000,127000,Diesel,Individual,Manual,First Owner,5,90.00,kmpl,23.00,1396
2,4,Hyundai,2017,440000,45000,Petrol,Individual,Manual,First Owner,5,81.86,kmpl,20.14,1197
3,7,Toyota,2011,350000,90000,Diesel,Individual,Manual,First Owner,5,67.10,kmpl,23.59,1364
4,8,Ford,2013,200000,169000,Diesel,Individual,Manual,First Owner,5,68.10,kmpl,20.00,1399


# 2. Inspección inicial (tipos, nulos, cardinalidad)

In [13]:
print("Tipos:\n", df.dtypes)
print("\nNulos:\n", df.isna().sum())


Tipos:
 Unnamed: 0              int64
name                   object
year                    int64
selling_price           int64
km_driven               int64
fuel                   object
seller_type            object
transmission           object
owner                  object
seats                   int64
max_power (in bph)    float64
Mileage Unit           object
Mileage               float64
Engine (CC)             int64
dtype: object

Nulos:
 Unnamed: 0            0
name                  0
year                  0
selling_price         0
km_driven             0
fuel                  0
seller_type           0
transmission          0
owner                 0
seats                 0
max_power (in bph)    0
Mileage Unit          0
Mileage               0
Engine (CC)           0
dtype: int64


# 3 Normalización de nombres y parseo básico

In [14]:
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

df.rename(columns={"unnamed:_0": "id"}, inplace=True)
df.columns

Index(['id', 'name', 'year', 'selling_price', 'km_driven', 'fuel',
       'seller_type', 'transmission', 'owner', 'seats', 'max_power_(in_bph)',
       'mileage_unit', 'mileage', 'engine_(cc)'],
      dtype='object')

# Separa tipos de variables

In [15]:
TARGET = "selling_price"

num_feats = df.select_dtypes(include="number").columns.drop(TARGET).tolist()
cat_feats = df.select_dtypes(include=["object","category","bool"]).columns.tolist()

print(f"Número de features numéricas: {len(num_feats)}")
print(f"Número de features categóricas: {len(cat_feats)}")

print(f"Features numéricas: {num_feats}")
print(f"Features categóricas: {cat_feats}")

Número de features numéricas: 7
Número de features categóricas: 6
Features numéricas: ['id', 'year', 'km_driven', 'seats', 'max_power_(in_bph)', 'mileage', 'engine_(cc)']
Features categóricas: ['name', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage_unit']


# 2 Train/Test split

In [22]:
from sklearn.model_selection import train_test_split

X = df[cat_feats + num_feats].copy()
y = df[TARGET].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=150
)
X_train.shape, X_test.shape

((1676, 13), (419, 13))

# Pipeline (imputación + One-Hot + escalado) + Regresión Lineal

In [16]:
from sklearn.compose import ColumnTransformer           
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


In [19]:
numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])


print(numeric_tf)
print(categorical_tf)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])
Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [20]:
preprocess = ColumnTransformer([
    ("num", numeric_tf, num_feats),
    ("cat", categorical_tf, cat_feats)
])

model = Pipeline([
    ("prep", preprocess),
    ("linreg", LinearRegression())
])

# Entrenar el modelo

In [23]:
model.fit(X_train, y_train)

,steps,"[('prep', ...), ('linreg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


# Evaluación (MAE, RMSE, R²) + Visualizaciones

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"MAE  (Error Absoluto Medio):       {mae:,.2f}  -> en promedio, cuánto se equivoca el modelo en las unidades de la variable objetivo.")
print(f"RMSE (Raíz del Error Cuadrático): {rmse:,.2f}  -> penaliza más los errores grandes, también en las unidades del objetivo.")
print(f"R²   (Coef. de Determinación):     {r2:.4f}   -> proporción de la variabilidad explicada por el modelo (1 = perfecto, 0 = no explica nada).")


MAE  (Error Absoluto Medio):       79,869.46  -> en promedio, cuánto se equivoca el modelo en las unidades de la variable objetivo.
RMSE (Raíz del Error Cuadrático): 11,520,322,945.23  -> penaliza más los errores grandes, también en las unidades del objetivo.
R²   (Coef. de Determinación):     0.7812   -> proporción de la variabilidad explicada por el modelo (1 = perfecto, 0 = no explica nada).


# Visualizaciones de las predicciones del modelo


In [25]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configurar el estilo de los gráficos
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# 1. Gráfico de dispersión: Valores Reales vs Predicciones
plt.figure(figsize=(10, 8))
plt.scatter(y_test, y_pred, alpha=0.6, color='blue', edgecolors='black', linewidth=0.5)

# Línea de predicción perfecta (y = x)
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Predicción Perfecta')

plt.xlabel('Valores Reales (Precio de Venta)', fontsize=12)
plt.ylabel('Predicciones del Modelo', fontsize=12)
plt.title('Comparación: Valores Reales vs Predicciones\nModelo de Regresión Lineal', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Agregar métricas en el gráfico
textstr = f'R² = {r2:.4f}\nMAE = {mae:,.0f}\nRMSE = {np.sqrt(rmse):,.0f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()


/var/folders/jk/2v8rfqkn4kd4wht4gmnrjlvw0000gn/T/ipykernel_26755/3985560857.py:32: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [26]:
# 2. Análisis de Residuos
residuos = y_test - y_pred

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico de residuos vs predicciones
axes[0, 0].scatter(y_pred, residuos, alpha=0.6, color='green', edgecolors='black', linewidth=0.5)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Predicciones')
axes[0, 0].set_ylabel('Residuos (Real - Predicción)')
axes[0, 0].set_title('Residuos vs Predicciones')
axes[0, 0].grid(True, alpha=0.3)

# Histograma de residuos
axes[0, 1].hist(residuos, bins=30, alpha=0.7, color='orange', edgecolor='black')
axes[0, 1].set_xlabel('Residuos')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Residuos')
axes[0, 1].grid(True, alpha=0.3)

# Q-Q plot de residuos
from scipy.stats import probplot
probplot(residuos, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot de Residuos')
axes[1, 0].grid(True, alpha=0.3)

# Residuos vs valores reales
axes[1, 1].scatter(y_test, residuos, alpha=0.6, color='purple', edgecolors='black', linewidth=0.5)
axes[1, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Valores Reales')
axes[1, 1].set_ylabel('Residuos')
axes[1, 1].set_title('Residuos vs Valores Reales')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Análisis de Residuos del Modelo de Regresión Lineal', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


/var/folders/jk/2v8rfqkn4kd4wht4gmnrjlvw0000gn/T/ipykernel_26755/1007573514.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [27]:
# 3. Distribución de Predicciones vs Valores Reales
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Histograma comparativo
axes[0].hist(y_test, bins=30, alpha=0.7, label='Valores Reales', color='blue', edgecolor='black')
axes[0].hist(y_pred, bins=30, alpha=0.7, label='Predicciones', color='red', edgecolor='black')
axes[0].set_xlabel('Precio de Venta')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución: Valores Reales vs Predicciones')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot comparativo
data_to_plot = [y_test, y_pred]
box_plot = axes[1].boxplot(data_to_plot, labels=['Valores Reales', 'Predicciones'], patch_artist=True)
box_plot['boxes'][0].set_facecolor('blue')
box_plot['boxes'][1].set_facecolor('red')
axes[1].set_ylabel('Precio de Venta')
axes[1].set_title('Boxplot: Comparación de Distribuciones')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Análisis de Distribuciones', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


/var/folders/jk/2v8rfqkn4kd4wht4gmnrjlvw0000gn/T/ipykernel_26755/3711495059.py:15: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  box_plot = axes[1].boxplot(data_to_plot, labels=['Valores Reales', 'Predicciones'], patch_artist=True)
/var/folders/jk/2v8rfqkn4kd4wht4gmnrjlvw0000gn/T/ipykernel_26755/3711495059.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [28]:
# 4. Visualización de Métricas de Rendimiento
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Métricas del modelo
metrics = ['MAE', 'RMSE', 'R²']
values = [mae, np.sqrt(rmse), r2]
colors = ['lightcoral', 'lightblue', 'lightgreen']

# Gráfico de barras de métricas
bars = axes[0].bar(metrics[:2], values[:2], color=colors[:2], edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Valor de la Métrica')
axes[0].set_title('Métricas de Error (MAE y RMSE)')
axes[0].grid(True, alpha=0.3, axis='y')

# Agregar valores en las barras
for bar, value in zip(bars, values[:2]):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + height*0.02,
                f'{value:,.0f}', ha='center', va='bottom', fontweight='bold')

# Gráfico separado para R² (escala diferente)
r2_bar = axes[1].bar(['R²'], [r2], color=[colors[2]], edgecolor='black', linewidth=1.5)
axes[1].set_ylabel('Coeficiente de Determinación')
axes[1].set_title('R² Score')
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].text(0, r2 + 0.02, f'{r2:.4f}', ha='center', va='bottom', fontweight='bold')

# Gráfico de error absoluto por rango de precios
# Dividir en quintiles por precio real
y_test_sorted = y_test.sort_values()
quintiles = np.quantile(y_test_sorted, [0.2, 0.4, 0.6, 0.8])
labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']

errors_by_quintile = []
for i in range(5):
    if i == 0:
        mask = y_test <= quintiles[0]
    elif i == 4:
        mask = y_test > quintiles[3]
    else:
        mask = (y_test > quintiles[i-1]) & (y_test <= quintiles[i])
    
    quintile_errors = np.abs(y_test[mask] - y_pred[mask])
    errors_by_quintile.append(quintile_errors.mean())

axes[2].bar(labels, errors_by_quintile, color='mediumpurple', edgecolor='black', linewidth=1.5)
axes[2].set_xlabel('Quintiles de Precio')
axes[2].set_ylabel('Error Absoluto Medio')
axes[2].set_title('MAE por Rango de Precios')
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(True, alpha=0.3, axis='y')

plt.suptitle('Resumen de Rendimiento del Modelo', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


/var/folders/jk/2v8rfqkn4kd4wht4gmnrjlvw0000gn/T/ipykernel_26755/1457476362.py:56: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [29]:
# 5. Importan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     cia de Características (Coeficientes del Modelo)
# Obtener los nombres de las características después del preprocesamiento
feature_names = []

# Características numéricas (después del escalado)
feature_names.extend(num_feats)

# Características categóricas (después del one-hot encoding)
onehot_encoder = model.named_steps['prep'].named_transformers_['cat'].named_steps['onehot']
cat_feature_names = onehot_encoder.get_feature_names_out(cat_feats)
feature_names.extend(cat_feature_names)

# Obtener los coeficientes del modelo de regresión lineal
coefficients = model.named_steps['linreg'].coef_

# Crear un DataFrame para facilitar la visualización
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

# Seleccionar las top 15 características más importantes
top_features = coef_df.head(15)

# Visualizar
plt.figure(figsize=(12, 8))
colors = ['red' if x < 0 else 'green' for x in top_features['coefficient']]
bars = plt.barh(range(len(top_features)), top_features['coefficient'], color=colors, alpha=0.7, edgecolor='black')

plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Coeficiente')
plt.ylabel('Características')
plt.title('Top 15 Características Más Importantes\n(Coeficientes del Modelo de Regresión Lineal)', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.grid(True, alpha=0.3, axis='x')

# Agregar leyenda
red_patch = plt.Rectangle((0,0),1,1, color='red', alpha=0.7, label='Impacto Negativo')
green_patch = plt.Rectangle((0,0),1,1, color='green', alpha=0.7, label='Impacto Positivo')
plt.legend(handles=[green_patch, red_patch], loc='lower right')

plt.tight_layout()
plt.show()

# Mostrar tabla con los valores
print("\\nTop 15 Características Más Importantes:")
print("="*50)
for idx, row in top_features.iterrows():
    impact = "📈 Positivo" if row['coefficient'] > 0 else "📉 Negativo"
    print(f"{row['feature']:<30} | Coef: {row['coefficient']:>10.2f} | {impact}")


\nTop 15 Características Más Importantes:
owner_Test Drive Car           | Coef:  475833.74 | 📈 Positivo
name_Honda                     | Coef:  181808.66 | 📈 Positivo
name_Chevrolet                 | Coef: -166670.38 | 📉 Negativo
name_Toyota                    | Coef:  143982.73 | 📈 Positivo
owner_Second Owner             | Coef: -138718.55 | 📉 Negativo
owner_Third Owner              | Coef: -127921.87 | 📉 Negativo
max_power_(in_bph)             | Coef:  122329.17 | 📈 Positivo
year                           | Coef:  113671.40 | 📈 Positivo
owner_Fourth & Above Owner     | Coef: -109433.72 | 📉 Negativo
name_Volkswagen                | Coef: -106930.67 | 📉 Negativo
owner_First Owner              | Coef:  -99759.60 | 📉 Negativo
name_Tata                      | Coef:  -92966.85 | 📉 Negativo
fuel_Diesel                    | Coef:   87451.17 | 📈 Positivo
fuel_Petrol                    | Coef:  -73283.77 | 📉 Negativo
name_Maruti                    | Coef:   48343.91 | 📈 Positivo


/var/folders/jk/2v8rfqkn4kd4wht4gmnrjlvw0000gn/T/ipykernel_26755/67099663.py:44: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# 6. Visualización Final: Línea de Regresión con Datos y Predicciones


In [30]:
# Para visualizar la regresión lineal necesitamos seleccionar una característica numérica principal
# Vamos a usar la característica más importante del modelo

# Encontrar la característica numérica más importante
numeric_coef_indices = list(range(len(num_feats)))
numeric_coefficients = coefficients[numeric_coef_indices]
numeric_importance = pd.DataFrame({
    'feature': num_feats,
    'coefficient': numeric_coefficients,
    'abs_coefficient': np.abs(numeric_coefficients)
}).sort_values('abs_coefficient', ascending=False)

most_important_feature = numeric_importance.iloc[0]['feature']
print(f"Característica numérica más importante: {most_important_feature}")

# Crear el gráfico de regresión lineal
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Primer gráfico: Característica más importante vs Precio real
feature_values = X_test[most_important_feature]

# Scatter plot de datos reales
axes[0].scatter(feature_values, y_test, alpha=0.6, color='blue', s=50, 
               edgecolors='darkblue', linewidth=0.5, label='Datos Reales')

# Calcular y dibujar línea de tendencia
z = np.polyfit(feature_values, y_test, 1)
p = np.poly1d(z)
x_line = np.linspace(feature_values.min(), feature_values.max(), 100)
axes[0].plot(x_line, p(x_line), "r--", linewidth=2, label=f'Línea de Tendencia')

axes[0].set_xlabel(f'{most_important_feature}', fontsize=12)
axes[0].set_ylabel('Precio de Venta Real', fontsize=12)
axes[0].set_title(f'Regresión Lineal: {most_important_feature} vs Precio Real', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Segundo gráfico: Predicciones vs Valores reales con línea de regresión perfecta
axes[1].scatter(y_test, y_pred, alpha=0.6, color='green', s=50, 
               edgecolors='darkgreen', linewidth=0.5, label='Predicciones del Modelo')

# Línea de regresión perfecta (y = x)
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
axes[1].plot([min_val, max_val], [min_val, max_val], 'r-', linewidth=3, 
            label='Línea de Regresión Perfecta (y=x)')

# Línea de regresión real de las predicciones
z_pred = np.polyfit(y_test, y_pred, 1)
p_pred = np.poly1d(z_pred)
axes[1].plot(y_test, p_pred(y_test), 'orange', linewidth=2, linestyle='--',
            label=f'Línea de Regresión Real (R²={r2:.3f})')

axes[1].set_xlabel('Valores Reales', fontsize=12)
axes[1].set_ylabel('Predicciones del Modelo', fontsize=12)
axes[1].set_title('Línea de Regresión: Predicciones vs Valores Reales', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Agregar estadísticas en el segundo gráfico
textstr = f'Ecuación: y = {z_pred[0]:.2f}x + {z_pred[1]:.0f}\nR² = {r2:.4f}\nMAE = {mae:,.0f}'
props = dict(boxstyle='round', facecolor='lightyellow', alpha=0.8)
axes[1].text(0.05, 0.95, textstr, transform=axes[1].transAxes, fontsize=10,
            verticalalignment='top', bbox=props)

plt.suptitle('Análisis de Regresión Lineal - Visualización Final', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\\n📊 Resumen de la Regresión Lineal:")
print(f"{'='*50}")
print(f"🔹 Característica más influyente: {most_important_feature}")
print(f"🔹 Coeficiente: {numeric_importance.iloc[0]['coefficient']:.2f}")
print(f"🔹 Ecuación de predicción vs real: y = {z_pred[0]:.2f}x + {z_pred[1]:.0f}")
print(f"🔹 R² del modelo: {r2:.4f} ({r2*100:.1f}% de varianza explicada)")
print(f"🔹 Error promedio: ±{mae:,.0f} unidades monetarias")


Característica numérica más importante: max_power_(in_bph)
\n📊 Resumen de la Regresión Lineal:
🔹 Característica más influyente: max_power_(in_bph)
🔹 Coeficiente: 122329.17
🔹 Ecuación de predicción vs real: y = 0.73x + 116552
🔹 R² del modelo: 0.7812 (78.1% de varianza explicada)
🔹 Error promedio: ±79,869 unidades monetarias


/var/folders/jk/2v8rfqkn4kd4wht4gmnrjlvw0000gn/T/ipykernel_26755/810221049.py:68: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [32]:
import joblib
import numpy as np

MODEL_PATH = "data/linreg_pipeline.pkl"

# Guardar
joblib.dump(model, MODEL_PATH)
print("Modelo guardado en:", MODEL_PATH)



Modelo guardado en: data/linreg_pipeline.pkl


In [ ]:
# 🎨 VISUALIZACIONES COMPLETAS - VERSIÓN ROBUSTA
# Esta celda contiene todas las visualizaciones en una versión que evita errores de backend

import warnings
warnings.filterwarnings('ignore')

# Configuración segura de matplotlib
import matplotlib
matplotlib.use('Agg')  # Backend que siempre funciona
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
    import seaborn as sns
    sns.set_palette("husl")
    USE_SEABORN = True
except:
    USE_SEABORN = False
    print("Seaborn no disponible, usando matplotlib básico")

# Configurar estilo básico
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("🎨 Iniciando visualizaciones del modelo de regresión lineal...")
print("="*60)

# 1. GRÁFICO PRINCIPAL: Valores Reales vs Predicciones
plt.figure(figsize=(10, 8))
plt.scatter(y_test, y_pred, alpha=0.6, color='steelblue', s=30, edgecolors='navy', linewidth=0.3)

# Línea de predicción perfecta
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2.5, label='Predicción Perfecta (y=x)')

plt.xlabel('Valores Reales (Precio de Venta)', fontsize=12, fontweight='bold')
plt.ylabel('Predicciones del Modelo', fontsize=12, fontweight='bold')
plt.title('Comparación: Valores Reales vs Predicciones\nModelo de Regresión Lineal', 
          fontsize=14, fontweight='bold', pad=20)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Agregar métricas en el gráfico
textstr = f'R² = {r2:.4f}\nMAE = {mae:,.0f}\nRMSE = {np.sqrt(rmse):,.0f}'
props = dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8, edgecolor='navy')
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=11,
         verticalalignment='top', bbox=props, fontweight='bold')

plt.tight_layout()
plt.savefig('predicciones_vs_reales.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

print("✅ Gráfico 1: Predicciones vs Valores Reales - Completado")

# 2. ANÁLISIS DE RESIDUOS
residuos = y_test - y_pred

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Residuos vs Predicciones
axes[0, 0].scatter(y_pred, residuos, alpha=0.6, color='forestgreen', s=25, edgecolors='darkgreen', linewidth=0.3)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Predicciones', fontweight='bold')
axes[0, 0].set_ylabel('Residuos (Real - Predicción)', fontweight='bold')
axes[0, 0].set_title('Residuos vs Predicciones', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Histograma de residuos
axes[0, 1].hist(residuos, bins=25, alpha=0.7, color='orange', edgecolor='darkorange', linewidth=1)
axes[0, 1].set_xlabel('Residuos', fontweight='bold')
axes[0, 1].set_ylabel('Frecuencia', fontweight='bold')
axes[0, 1].set_title('Distribución de Residuos', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Residuos vs Valores Reales  
axes[1, 0].scatter(y_test, residuos, alpha=0.6, color='purple', s=25, edgecolors='indigo', linewidth=0.3)
axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Valores Reales', fontweight='bold')
axes[1, 0].set_ylabel('Residuos', fontweight='bold')
axes[1, 0].set_title('Residuos vs Valores Reales', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Estadísticas de residuos
residuos_stats = f'Media: {np.mean(residuos):,.0f}\nDesv. Est.: {np.std(residuos):,.0f}\nMin: {np.min(residuos):,.0f}\nMax: {np.max(residuos):,.0f}'
axes[1, 1].text(0.1, 0.7, residuos_stats, transform=axes[1, 1].transAxes, fontsize=12,
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8),
                verticalalignment='top', fontweight='bold')
axes[1, 1].set_title('Estadísticas de Residuos', fontweight='bold')
axes[1, 1].axis('off')

plt.suptitle('Análisis Completo de Residuos', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('analisis_residuos.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

print("✅ Gráfico 2: Análisis de Residuos - Completado")

# 3. DISTRIBUCIONES COMPARATIVAS
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Histogramas superpuestos
axes[0].hist(y_test, bins=25, alpha=0.7, label='Valores Reales', color='dodgerblue', edgecolor='navy')
axes[0].hist(y_pred, bins=25, alpha=0.7, label='Predicciones', color='crimson', edgecolor='darkred')
axes[0].set_xlabel('Precio de Venta', fontweight='bold')
axes[0].set_ylabel('Frecuencia', fontweight='bold')
axes[0].set_title('Distribuciones: Reales vs Predicciones', fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Boxplots comparativos
data_boxplot = [y_test.values, y_pred]
box_plot = axes[1].boxplot(data_boxplot, labels=['Valores Reales', 'Predicciones'], 
                          patch_artist=True, widths=0.6)
box_plot['boxes'][0].set_facecolor('dodgerblue')
box_plot['boxes'][0].set_alpha(0.7)
box_plot['boxes'][1].set_facecolor('crimson')
box_plot['boxes'][1].set_alpha(0.7)
axes[1].set_ylabel('Precio de Venta', fontweight='bold')
axes[1].set_title('Comparación de Distribuciones (Boxplot)', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('distribuciones_comparativas.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

print("✅ Gráfico 3: Distribuciones Comparativas - Completado")

print("\\n🎯 RESUMEN DEL ANÁLISIS VISUAL:")
print("="*50)
print(f"📊 R² (Bondad de ajuste): {r2:.4f} ({r2*100:.1f}% de varianza explicada)")
print(f"📏 MAE (Error absoluto medio): {mae:,.0f} unidades")
print(f"📐 RMSE (Error cuadrático medio): {np.sqrt(rmse):,.0f} unidades")
print(f"🎯 Desviación estándar de residuos: {np.std(residuos):,.0f}")
print("\\n✅ Todas las visualizaciones completadas exitosamente!")


🎨 Iniciando visualizaciones del modelo de regresión lineal...
✅ Gráfico 1: Predicciones vs Valores Reales - Completado
✅ Gráfico 2: Análisis de Residuos - Completado
✅ Gráfico 3: Distribuciones Comparativas - Completado
\n🎯 RESUMEN DEL ANÁLISIS VISUAL:
📊 R² (Bondad de ajuste): 0.7812 (78.1% de varianza explicada)
📏 MAE (Error absoluto medio): 79,869 unidades
📐 RMSE (Error cuadrático medio): 107,333 unidades
🎯 Desviación estándar de residuos: 107,300
\n✅ Todas las visualizaciones completadas exitosamente!


In [ ]:
print("\\n" + "="*60)
print("🎯 GRÁFICO ESPECIAL: LÍNEA DE REGRESIÓN LINEAL")
print("="*60)

# Para mostrar la línea de regresión, seleccionamos la característica numérica más importante
# Primero identificamos cuál es la característica más influyente

# Obtener los coeficientes para características numéricas
try:
    # Obtener nombres de características después del preprocesamiento
    feature_names = []
    feature_names.extend(num_feats)
    
    # Características categóricas (one-hot encoding)
    onehot_encoder = model.named_steps['prep'].named_transformers_['cat'].named_steps['onehot']
    cat_feature_names = onehot_encoder.get_feature_names_out(cat_feats)
    feature_names.extend(cat_feature_names)
    
    # Coeficientes del modelo
    coefficients = model.named_steps['linreg'].coef_
    
    # Encontrar la característica numérica más importante
    numeric_coef_indices = list(range(len(num_feats)))
    numeric_coefficients = coefficients[numeric_coef_indices]
    numeric_importance = pd.DataFrame({
        'feature': num_feats,
        'coefficient': numeric_coefficients,
        'abs_coefficient': np.abs(numeric_coefficients)
    }).sort_values('abs_coefficient', ascending=False)
    
    most_important_feature = numeric_importance.iloc[0]['feature']
    print(f"📈 Característica más importante: {most_important_feature}")
    print(f"📊 Coeficiente: {numeric_importance.iloc[0]['coefficient']:.3f}")
    
except Exception as e:
    # Si hay problemas, usar una característica por defecto
    most_important_feature = 'year'  # Variable que típicamente existe
    print(f"⚠️  Usando característica por defecto: {most_important_feature}")

# CREAR LA VISUALIZACIÓN CON LÍNEA DE REGRESIÓN
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# GRÁFICO 1: Característica vs Precio Real con Línea de Regresión
if most_important_feature in X_test.columns:
    feature_values = X_test[most_important_feature].values
    
    # Scatter plot de datos reales
    axes[0].scatter(feature_values, y_test, alpha=0.6, color='royalblue', s=40, 
                   edgecolors='navy', linewidth=0.5, label='Datos Reales')
    
    # Calcular y dibujar línea de regresión
    z = np.polyfit(feature_values, y_test, 1)
    p = np.poly1d(z)
    
    # Crear línea suave
    x_line = np.linspace(feature_values.min(), feature_values.max(), 100)
    y_line = p(x_line)
    
    axes[0].plot(x_line, y_line, "red", linewidth=3, 
                label=f'Línea de Regresión\\ny = {z[0]:.2f}x + {z[1]:.0f}')
    
    # Calcular R² para esta característica individual
    y_pred_single = p(feature_values)
    r2_single = r2_score(y_test, y_pred_single)
    
    axes[0].set_xlabel(f'{most_important_feature}', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Precio de Venta Real', fontsize=12, fontweight='bold')
    axes[0].set_title(f'Regresión Lineal Simple\\n{most_important_feature} vs Precio', 
                     fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    
    # Agregar estadísticas
    stats_text = f'R² = {r2_single:.4f}\\nPendiente = {z[0]:.2f}\\nIntercepto = {z[1]:.0f}'
    axes[0].text(0.05, 0.95, stats_text, transform=axes[0].transAxes, fontsize=10,
                bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.8),
                verticalalignment='top', fontweight='bold')

# GRÁFICO 2: Predicciones vs Reales con Líneas de Regresión
axes[1].scatter(y_test, y_pred, alpha=0.6, color='mediumseagreen', s=40, 
               edgecolors='darkgreen', linewidth=0.5, label='Predicciones del Modelo')

# Línea de predicción perfecta (y = x)
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
axes[1].plot([min_val, max_val], [min_val, max_val], 'red', linewidth=4, 
            label='Línea Ideal (y = x)', alpha=0.8)

# Línea de regresión real entre predicciones y valores reales
z_pred = np.polyfit(y_test, y_pred, 1)
p_pred = np.poly1d(z_pred)
y_test_sorted = np.sort(y_test)
axes[1].plot(y_test_sorted, p_pred(y_test_sorted), 'orange', linewidth=3, 
            linestyle='--', label=f'Regresión Real\\ny = {z_pred[0]:.3f}x + {z_pred[1]:.0f}')

axes[1].set_xlabel('Valores Reales', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Predicciones del Modelo', fontsize=12, fontweight='bold')
axes[1].set_title('Líneas de Regresión\\nPredicciones vs Valores Reales', 
                 fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

# Estadísticas del modelo completo
model_stats = f'R² Modelo = {r2:.4f}\\nMAE = {mae:,.0f}\\nRMSE = {np.sqrt(rmse):,.0f}'
axes[1].text(0.05, 0.95, model_stats, transform=axes[1].transAxes, fontsize=10,
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
            verticalalignment='top', fontweight='bold')

plt.suptitle('🎯 ANÁLISIS FINAL: LÍNEAS DE REGRESIÓN LINEAL', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('lineas_regresion_final.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

print("\\n🎉 VISUALIZACIÓN FINAL COMPLETADA")
print("="*50)
print("📊 Gráficos generados:")
print("   1️⃣ Regresión simple (característica individual)")
print("   2️⃣ Regresión del modelo completo")
print("   ✅ Líneas rojas muestran las tendencias lineales")
print("   ✅ Archivos PNG guardados para referencia")
print("\\n🎯 ¡Análisis completo de regresión lineal finalizado!")


# Cargar Modelo

In [34]:
# Cargar
import joblib
model_2 = joblib.load(MODEL_PATH)
print("Modelo cargado.")


Modelo cargado.


In [35]:
import pandas as pd

sample1 = {
    "engine_(cc)": 1197,
    "seats": 5,
    "mileage": 18,       
    "mileage_num": 18.0,       
    "id": 1,                   
    "max_power_(in_bph)": 82.0,
    "mileage_unit": "kmpl",
    "name": "Maruti Swift VDI",
    "fuel": "Petrol",
    "transmission": "Manual",
    "seller_type": "Individual",
    "owner": "First Owner",
    "year": 2020,
    "km_driven": 15000
}

sample2 = {
    "engine_(cc)": 1498,
    "seats": 5,
    "mileage": 22,
    "mileage_num": 22.0,
    "id": 2,
    "max_power_(in_bph)": 98.0,
    "mileage_unit": "kmpl",
    "name": "Honda City VX",
    "fuel": "Diesel",
    "transmission": "Automatic",
    "seller_type": "Dealer",
    "owner": "Second Owner",
    "year": 2018,
    "km_driven": 60000
}

new_df = pd.DataFrame([sample1, sample2])
print("Ejemplos manuales preparados:")
display(new_df)
print("Predicciones:")
print(model_2.predict(new_df))

Ejemplos manuales preparados:


,engine_(cc),seats,mileage,mileage_num,id,max_power_(in_bph),mileage_unit,name,fuel,transmission,seller_type,owner,year,km_driven
0,1197,5,18,18.0,1,82.0,kmpl,Maruti Swift VDI,Petrol,Manual,Individual,First Owner,2020,15000
1,1498,5,22,22.0,2,98.0,kmpl,Honda City VX,Diesel,Automatic,Dealer,Second Owner,2018,60000


Predicciones:
[688132.07535066 742010.21298382]
